In [4]:
# Load libraries
from sklearn.cross_validation import train_test_split
import numpy
import pandas

# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']

dataset = pandas.read_csv(url, names=names)

# Not Sure
data = numpy.array(dataset)  #convert array to numpy type array
x_train ,x_test = train_test_split(data,test_size=0.20)

# shape
print(dataset.shape)


(1728, 6)
